In [35]:
#Geração dos Arquivos TXT

import os
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from keybert import KeyBERT
from DownloadDOEs import Baixar_DOEs
from Funções import extrair_orgaos_PDF, extrair_texto_entre_orgaos

# Downloads necessários do NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Caminhos para pastas de texto
txt_folder = 'txt_extraidos'

# Cria uma pasta para os arquivos TXT se não existir
if not os.path.exists(txt_folder):
    os.makedirs(txt_folder)

# Função para pré-processamento de texto
def preprocess_text(text):
    stop_words = set(stopwords.words('portuguese'))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

# Função para extração de palavras-chave usando KeyBERT
def extract_keywords(text):
    kw_model = KeyBERT()
    try:
        keywords = kw_model.extract_keywords(text, top_n=20)
        return [kw[0] for kw in keywords]  # Retorna apenas a palavra-chave (str)
    except IndexError:
        return []  # Retorna uma lista vazia se não houver palavras-chave extraídas

# Função principal para processar os documentos PDF e gerar arquivos TXT
def gerar_arquivos_TXT():
    # Cria uma pasta com o nome especificado e baixa os Diários Oficiais do Estado até número especificado dias atrás
    Baixar_DOEs('DOEs', 2)

    # Pasta que guarda quais DOEs já estão extraídos em .json
    if not os.path.exists('DOEsExtraidos'):
        os.makedirs('DOEsExtraidos')

    print('Após baixado os DOEs, inicia a extração:\n')

    # Essa parte faz toda a extração de órgãos e conteúdo dos documentos baixados
    pdfpasta = os.listdir('DOEs/')
    pdfpasta.sort()

    for P in pdfpasta:
        if os.path.exists(os.path.join('DOEsExtraidos', P)):
            print(P + ' Documento já utilizado')
            os.remove(os.path.join('DOEs', P))
        elif P.endswith('.pdf'):
            print('Documento ' + P)
            z = 'DOEs/' + P
            bloco = z
            bloco = bloco.replace('DOEs/do', '')
            bloco = bloco.split('p')
            caderno = bloco[1]
            bloco = bloco[0]
            caderno = caderno.replace('0', '')
            caderno = caderno.replace('.', '')
            caderno = int(caderno)
            print(bloco, caderno)
            datachar = bloco[6:8]
            meschar = bloco[4:6]
            anochar = bloco[0:4]

            temp = f'{datachar}-{meschar}-{anochar}'  # Definir a data no formato correto

            # Essa parte extrai os órgãos e conteúdo dos documentos baixados
            listadocs = extrair_orgaos_PDF(z)
            listacontextos = extrair_texto_entre_orgaos(listadocs)

            listX = []

            for c in range(0, len(listacontextos)):
                if '(Continuação)' in listacontextos[c].nome or ' (Continuação)' in listacontextos[c].nome:
                    listacontextos[c].nome = listacontextos[c].nome.replace(' (Continuação)', '')
                    if isinstance(listacontextos[c].publicacao, list):
                        for T in range(0, len(listacontextos[c].publicacao)):
                            listX.append({
                                'DATA': temp,
                                'CADERNO': caderno,
                                'PAGINA': listacontextos[c].publicacao[T].page1,
                                'NOME': listacontextos[c].nome,
                                'PUBLICACAO': 1 + T,
                                'TITULO': None,
                                'ASSUNTO': None,
                                'TEXTO': listacontextos[c].publicacao[T].texto,
                                'LISTANEGRITO': listacontextos[c].publicacao[T].negrito
                            })
                    else:
                        listX.append({
                            'DATA': temp,
                            'CADERNO': caderno,
                            'PAGINA': listacontextos[c].publicacao.page1,
                            'NOME': listacontextos[c].nome,
                            'PUBLICACAO': 1 + T,
                            'TITULO': None,
                            'ASSUNTO': None,
                            'TEXTO': listacontextos[c].publicacao.texto,
                            'LISTANEGRITO': listacontextos[c].publicacao.negrito
                        })
                else:
                    if isinstance(listacontextos[c].publicacao, list):
                        for T in range(0, len(listacontextos[c].publicacao)):
                            listX.append({
                                'DATA': temp,
                                'CADERNO': caderno,
                                'PAGINA': listacontextos[c].publicacao[T].page1,
                                'NOME': listacontextos[c].nome,
                                'PUBLICACAO': 1 + T,
                                'TITULO': None,
                                'ASSUNTO': None,
                                'TEXTO': listacontextos[c].publicacao[T].texto,
                                'LISTANEGRITO': listacontextos[c].publicacao[T].negrito
                            })
                    else:
                        listX.append({
                            'DATA': temp,
                            'CADERNO': caderno,
                            'PAGINA': listacontextos[c].publicacao.page1,
                            'NOME': listacontextos[c].nome,
                            'PUBLICACAO': 1 + T,
                            'TITULO': None,
                            'ASSUNTO': None,
                            'TEXTO': listacontextos[c].publicacao.texto,
                            'LISTANEGRITO': listacontextos[c].publicacao.negrito
                        })

            os.rename(os.path.join('DOEs', P), os.path.join('DOEsExtraidos', P))

            if temp != '':
                # Salvar listX em um arquivo TXT antes de prosseguir com a criação do JSON
                txt_file_path = os.path.join(txt_folder, f'{temp}.txt')
                with open(txt_file_path, 'w') as write_file:
                    for item in listX:
                        write_file.write(json.dumps(item) + '\n')

                print('Dados TXT salvos em:', txt_file_path)

# Chamar a função para gerar os arquivos TXT
gerar_arquivos_TXT()

[nltk_data] Downloading package punkt to
[nltk_data]     /user/home/bruno.rogerio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /user/home/bruno.rogerio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /user/home/bruno.rogerio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Diretório DOEs já existe.
PDF ja existe: do20240703p01.pdf
PDF ja existe: do20240703p02.pdf
PDF ja existe: do20240703p03.pdf
PDF ja existe: do20240702p01.pdfPDF ja existe: do20240702p02.pdf

Após baixado os DOEs, inicia a extração:

Documento do20240702p01.pdf
20240702 1
Dados TXT salvos em: txt_extraidos/02-07-2024.txt
Documento do20240702p02.pdf
20240702 2
Dados TXT salvos em: txt_extraidos/02-07-2024.txt
Documento do20240703p01.pdf
20240703 1
Dados TXT salvos em: txt_extraidos/03-07-2024.txt
Documento do20240703p02.pdf
20240703 2
Dados TXT salvos em: txt_extraidos/03-07-2024.txt
Documento do20240703p03.pdf
20240703 3
Dados TXT salvos em: txt_extraidos/03-07-2024.txt


In [6]:
pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 233.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 229.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 6.4 MB/s eta 0:00:00:00:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [31]:
#Gerar Arquivos JSON

import os
import re
import json
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from keybert import KeyBERT
from nltk.corpus import stopwords
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.stats import mode

# Downloads necessários do NLTK para uso posterior
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Definição dos caminhos para as pastas de arquivos TXT e JSON
txt_folder = 'txt_extraidos'
json_folder = 'json_extraidos'

# Cria a pasta para os arquivos JSON se ela não existir
if not os.path.exists(json_folder):
    os.makedirs(json_folder)

# Função para selecionar as palavras-chave mais representativas de um cluster
def get_representative_keywords(cluster_labels, keywords):
    clusters = {}
    # Agrupa palavras-chave por rótulos de cluster
    for idx, label in enumerate(cluster_labels):
        if label not in clusters:
            clusters[label] = []
        if idx < len(keywords):
            clusters[label].append(keywords[idx])
    
    # Encontra o cluster mais comum
    most_common_cluster = max(clusters, key=lambda k: len(clusters[k]))
    representative_keywords = clusters[most_common_cluster]
    return representative_keywords

# Função principal para processar cada arquivo TXT
def processar_arquivo_txt(txt_file):
    temp = os.path.basename(txt_file).replace('.txt', '')

    # Carrega os dados do arquivo TXT
    listX = []
    with open(txt_file, 'r') as read_file:
        for i, line in enumerate(read_file):
            pub = json.loads(line.strip())
            pub['TITULO'] = ""
            pub['ASSUNTO'] = ""
            listX.append(pub)

    # Define o título e o assunto para cada publicação
    for pub in listX:
        texto = pub.get('TEXTO', '')
        negrito = pub.get('LISTANEGRITO', [])

        # Pré-processamento do texto
        stop_words = set(stopwords.words('portuguese'))
        vectorizer = TfidfVectorizer(stop_words=list(stop_words))

        # Dividir o texto em trechos menores (sentenças)
        trechos = nltk.sent_tokenize(texto)
        
        # Verifica se há pelo menos dois trechos
        if len(trechos) < 2:
            pub['TITULO'] = "N/A"
            pub['ASSUNTO'] = "N/A"
            continue  # Pula para a próxima publicação se não houver trechos suficientes

        # Vetoriza todos os trechos juntos
        X_text = vectorizer.fit_transform(trechos)

        # Normalização Min-Max dos dados TF-IDF
        scaler = MinMaxScaler()
        X_normalized = scaler.fit_transform(X_text.toarray())

        # Define os métodos de clusterização
        kmeans = KMeans(n_clusters=2, init='k-means++', random_state=42)
        dbscan = DBSCAN(eps=0.1, min_samples=2)
        agg_clustering = AgglomerativeClustering(n_clusters=2, linkage='ward')

        # Aplica clusterização nos vetores TF-IDF normalizados dos trechos
        cluster_labels_kmeans = kmeans.fit_predict(X_normalized)
        cluster_labels_dbscan = dbscan.fit_predict(X_normalized)
        cluster_labels_hierarchical = agg_clustering.fit_predict(X_normalized)

        # Combina rótulos usando abordagem de votação
        combined_labels = np.vstack((cluster_labels_kmeans, cluster_labels_dbscan, cluster_labels_hierarchical))
        final_labels, _ = mode(combined_labels, axis=0)

        # Filtra trechos que estão em negrito
        filtered_final_rep_keywords = [kw for kw in trechos if any(neg in kw for neg in negrito)]
        
        # Ordena as palavras-chave representativas pela importância usando KeyBERT
        kw_model = KeyBERT()
        importance_final = []
        for kw in filtered_final_rep_keywords:
            keywords = kw_model.extract_keywords(''.join(kw), top_n=1)
            if keywords:
                importance_final.append(keywords[0][1])
            else:
                importance_final.append(0.0)  # Valor padrão para casos vazios

        sorted_importance_final = sorted(importance_final, reverse=True)
        
        sorted_final_keywords = [kw for _, kw in sorted(zip(sorted_importance_final, filtered_final_rep_keywords), reverse=True)]
        
        # Define o Título e o Assunto para o texto completo
        if sorted_final_keywords:
            pub['TITULO'] = ''.join(re.sub(r'\([^)]*\)', '', sorted_final_keywords[0]))  
            if len(sorted_final_keywords) > 1:
                pub['ASSUNTO'] = ''.join(re.sub(r'\([^)]*\)', '', sorted_final_keywords[1]))
            else:
                pub['ASSUNTO'] = "N/A"
        else:
            pub['TITULO'] = "N/A"
            pub['ASSUNTO'] = "N/A"

        # Remove o campo LISTANEGRITO antes de salvar
        if 'LISTANEGRITO' in pub:
            del pub['LISTANEGRITO']

    # Salva os resultados em arquivos JSON
    json_filename = os.path.join(json_folder, f"{temp}.json")
    with open(json_filename, 'w', encoding='utf-8') as json_file:
        json.dump(listX, json_file, ensure_ascii=False, indent=4)

    print(f"Resultados salvos em: {json_filename}")

# Chama a função principal para processar os arquivos TXT e gerar os arquivos JSON
txt_files = [os.path.join(txt_folder, f) for f in os.listdir(txt_folder) if f.endswith('.txt')]
for txt_file in txt_files:
    processar_arquivo_txt(txt_file)

[nltk_data] Downloading package punkt to
[nltk_data]     /user/home/bruno.rogerio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /user/home/bruno.rogerio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /user/home/bruno.rogerio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Resultados salvos em: json_extraidos/26-06-2024.json
